In [1]:
import pandas as pd
import numpy as np

def load_raceline_start(path):
    df = pd.read_csv(path)
    
    # 보통 포맷: t,x,y,yaw,speed 또는 x,y,heading...
    # 너가 준 raceline 파일은 x,y 좌표가 있는 걸로 가정.
    # 가장 앞 row를 start pose로 사용.
    x0 = df.iloc[0]["x"]
    y0 = df.iloc[0]["y"]
    
    return float(x0), float(y0)

raceline_start = load_raceline_start("./raceline_playground.csv")
print("Raceline Start =", raceline_start)

def is_log_matching_raceline(log_path, raceline_start, pos_tol=1.5):
    """
    log_path : RunLogger CSV
    raceline_start: (x0, y0)
    pos_tol : 출발 위치 판단 허용 거리 (m)
    """
    try:
        df = pd.read_csv(log_path)
        # 첫 row
        x = df.iloc[0]["x"]
        y = df.iloc[0]["y"]
    except Exception as e:
        print("[WARN] CSV 파싱 실패:", log_path)
        return False
    
    dx = float(x) - raceline_start[0]
    dy = float(y) - raceline_start[1]
    dist = np.hypot(dx, dy)
    
    return dist <= pos_tol


Raceline Start = (-10.0, 7.475)


In [2]:
import os
import re
import io
import math
import numpy as np
import pandas as pd

log_dir = "./logs/fgm"   # 필요하면 여기만 바꿔

# 파일명 패턴: Opp_bumper..._Gap..._Bub..._SPD..._FGM_..._dur_...s.csv
log_pattern = re.compile(
    r"^Opp_.*"                     # Opp_로 시작
    r"_FFG[-\d.]+"                 # _FFG<fgm_fov_angle>
    r"_SCD[-\d.]+"                 # _SCD<fgm_speed_check_fov_deg>
    r"_Gap[-\d.]+"                 # _Gap<...>
    r"_Bub[-\d.]+"                 # _Bub<...>
    r"_Clr[-\d.]+"                 # _Clr<...>
    r"_WW[-\d.]+"                  # _WW<...>
    r"_AW[-\d.]+"                  # _AW<...>
    r"_SW[-\d.]+"                  # _SW<...>
    r"_HB[-\d.]+"                  # _HB<...>
    r"_CT[-\d.]+"                  # _CT<...>
    r"_SA[-\d.]+"                  # _SA<...>
    r"_DBS[-\d.]+"                 # _DBS<...>
    r"_SPD[-\d.]+"                 # _SPD<...>
    r"_FGM_.*_dur_[-\d.]+s\.csv$"  # _FGM_<...>_dur_<t>s.csv
)

all_files = [
    os.path.join(log_dir, f)
    for f in os.listdir(log_dir)
    if f.endswith(".csv") and log_pattern.match(f)
]

print(f"발견된 로그 파일 개수: {len(all_files)}")
for f in all_files[:5]:
    print(" -", os.path.basename(f))

filtered_files = []
for f in all_files:
    if is_log_matching_raceline(f, raceline_start, pos_tol=1.5):
        filtered_files.append(f)
    else:
        print("❌ [FILTERED] Start position mismatch:", f)

print(f"✓ Valid logs: {len(filtered_files)} / {len(all_files)}")


발견된 로그 파일 개수: 423
 - Opp_bumper_v_2.csv_FFG185.3114_SCD27.2891_Gap0.8545_Bub0.4119_Clr0.4937_WW0.5558_AW8.8941_SW0.0570_HB1.8140_CT0.1837_SA0.5426_DBS0.1567_SPD4.3449_FGM_1764882966_GOAL_dur_11.76s.csv
 - Opp_bumper_v_5.csv_FFG179.5_SCD24.0_Gap0.870_Bub0.420_Clr0.500_WW0.525_AW7.55_SW0.064_HB1.75_CT0.200_SA0.540_DBS0.150_SPD4.30_FGM_1764880122_GOAL_dur_9.57s.csv
 - Opp_bumper_v_3.csv_FFG185.4_SCD24.1_Gap0.895_Bub0.442_Clr0.515_WW0.585_AW7.28_SW0.066_HB1.78_CT0.205_SA0.500_DBS0.147_SPD4.35_FGM_1764878243_GOAL_dur_9.94s.csv
 - Opp_bumper_slow_1.5.csv_FFG176.9_SCD26.0_Gap0.915_Bub0.470_Clr0.545_WW0.600_AW6.95_SW0.070_HB1.98_CT0.210_SA0.515_DBS0.140_SPD4.10_FGM_1764878863_ABORT_dur_0.00s.csv
 - Opp_bumper_slow_2.csv_FFG188.8425_SCD25.4270_Gap0.9078_Bub0.3976_Clr0.5093_WW0.5299_AW7.6821_SW0.0685_HB1.7813_CT0.2109_SA0.5429_DBS0.1535_SPD4.5172_FGM_1764882272_GOAL_dur_12.87s.csv
❌ [FILTERED] Start position mismatch: ./logs/fgm\Opp_bumper_slow_1.5.csv_FFG176.9_SCD26.0_Gap0.915_Bub0.470_Clr0.545

In [3]:
def load_run_log(path: str):
    """
    하나의 run_logger csv 파일을
    - df_data: per-step 데이터 (t,x,y,...)
    - summary_dict: 맨 아래 요약 블록 (key,value)
    로 나눠서 반환.
    """
    with open(path, "r") as f:
        lines = f.readlines()
    
    # Summary 시작 라인 찾기
    summary_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith('--- Experiment Summary'):
            summary_idx = i
            break
    
    if summary_idx is None:
        # Summary 없는 경우 (비상용)
        data_text = "".join(lines)
        df = pd.read_csv(io.StringIO(data_text))
        summary = {}
    else:
        # 맨 위 ~ summary 직전 빈 줄까지만 데이터로 사용
        data_text = "".join(lines[:summary_idx-1])
        df = pd.read_csv(io.StringIO(data_text))
        
        # summary 부분은 "key,value" 형식
        summary_lines = [l.strip() for l in lines[summary_idx+1:] if l.strip()]
        summary = {}
        for l in summary_lines:
            if "," in l:
                k, v = l.split(",", 1)
                summary[k.strip()] = v.strip()
    
    return df, summary


In [4]:
def parse_params_from_filename(path: str):
    """
    파일 이름에서
      scenario, Gap, Bub, Clr, WW, AW, SW, HB, CT, SA, DBS, SPD, result, duration_from_name
    를 뽑아서 dict로 반환.
    """
    name = os.path.basename(path)
    
    # 파라미터 블럭 파싱
    param_pattern = re.compile(
        r"_FFG(?P<FFG>[-\d.]+)"
        r"_SCD(?P<SCD>[-\d.]+)"
        r"_Gap(?P<Gap>[-\d.]+)"
        r"_Bub(?P<Bub>[-\d.]+)"
        r"_Clr(?P<Clr>[-\d.]+)"
        r"_WW(?P<WW>[-\d.]+)"
        r"_AW(?P<AW>[-\d.]+)"
        r"_SW(?P<SW>[-\d.]+)"
        r"_HB(?P<HB>[-\d.]+)"
        r"_CT(?P<CT>[-\d.]+)"
        r"_SA(?P<SA>[-\d.]+)"
        r"_DBS(?P<DBS>[-\d.]+)"
        r"_SPD(?P<SPD>[-\d.]+)"
    )

    
    m = param_pattern.search(name)
    params = {}
    if m:
        for k, v in m.groupdict().items():
            try:
                params[k] = float(v)
            except ValueError:
                params[k] = None
    
    # scenario (Gap 앞부분 전체)
    scen = name.split("_Gap")[0]
    params["scenario"] = scen   # ex) Opp_bumper_slow_0.5.csv
    
    # 결과 태그 (GOAL / CRASH / STUCK / ABORT 등)
    m2 = re.search(r"_(GOAL|CRASH|STUCK|ABORT)_", name)
    if m2:
        params["result"] = m2.group(1)
    
    # 파일명에 있는 duration
    m3 = re.search(r"_dur_([0-9.]+)s\.csv$", name)
    if m3:
        try:
            params["duration_from_name"] = float(m3.group(1))
        except ValueError:
            params["duration_from_name"] = None
    
    return params


In [5]:
def parse_summary_types(summary: dict):
    """
    summary dict 의 value를 적당히 숫자/불리언으로 캐스팅.
    """
    out = {}
    for k, v in summary.items():
        v = v.strip()
        # 숫자 시도
        try:
            out[k] = float(v)
            continue
        except ValueError:
            pass
        
        # YES/NO -> 1/0
        if v.upper() in ("YES", "NO"):
            out[k] = 1 if v.upper() == "YES" else 0
        else:
            out[k] = v
    return out


In [6]:
def compute_metrics(df: pd.DataFrame, safe_dist: float = 0.5):
    """
    run_logger 데이터 df 에서
    - total_time
    - collision_flag
    - min_dist / unsafe_ratio
    - fgm_min_dist / fgm_unsafe_ratio / fgm_track_rms
    를 계산 (있으면).
    """
    metrics = {}
    
    # 총 시간
    if "t" in df.columns and len(df) > 1:
        metrics["total_time"] = float(df["t"].iloc[-1] - df["t"].iloc[0])
    else:
        metrics["total_time"] = 0.0
    
    # 충돌 플래그
    if "collision" in df.columns:
        metrics["collision_flag"] = int(df["collision"].max() > 0)
    else:
        metrics["collision_flag"] = 0
    
    # 전역 min_d / unsafe_ratio
    if "min_d" in df.columns:
        metrics["min_dist"] = float(df["min_d"].min())
        metrics["unsafe_ratio"] = float((df["min_d"] < safe_dist).mean())
    
    # FGM 전용
    if "planner" in df.columns and "fgm_min_d" in df.columns:
        mask_fgm = df["planner"].astype(str).str.contains("FGM")
        if mask_fgm.any():
            df_fgm = df[mask_fgm]
            metrics["fgm_min_dist"] = float(df_fgm["fgm_min_d"].min())
            metrics["fgm_unsafe_ratio"] = float((df_fgm["fgm_min_d"] < safe_dist).mean())
            if "fgm_track" in df_fgm.columns:
                metrics["fgm_track_rms"] = float(math.sqrt((df_fgm["fgm_track"] ** 2).mean()))
    
    return metrics


In [7]:
def analyze_log(path: str, safe_dist: float = 0.5):
    """
    하나의 로그 파일 경로를 받아서
    - 파일 이름 기반 파라미터
    - summary 블럭 내용
    - per-step metrics
    다 합쳐서 dict 로 리턴.
    """
    df, summary = load_run_log(path)
    params   = parse_params_from_filename(path)
    metrics  = compute_metrics(df, safe_dist=safe_dist)
    summary_parsed = parse_summary_types(summary)
    
    row = {}
    row.update(params)
    row.update(summary_parsed)
    row.update(metrics)
    row["log_path"] = path
    
    return row


In [8]:
rows = []
for p in filtered_files:
    row = analyze_log(p, safe_dist=0.5)
    rows.append(row)

summary_df = pd.DataFrame(rows)
print("정리된 행 개수:", len(summary_df))
summary_df.head()


정리된 행 개수: 385


,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,HB,CT,...,Min Dist (m),Unsafe Ratio,FGM Min Dist (m),FGM Unsafe Ratio,FGM Track RMS (m),total_time,collision_flag,min_dist,unsafe_ratio,log_path
0,185.3114,27.2891,0.8545,0.4119,0.4937,0.5558,8.8941,0.0570,1.8140,0.1837,...,0.0,1.0,1.000000e+09,0.0,0.0,11.755911,0,0.0,1.0,./logs/fgm\Opp_bumper_v_2.csv_FFG185.3114_SCD2...
1,179.5000,24.0000,0.8700,0.4200,0.5000,0.5250,7.5500,0.0640,1.7500,0.2000,...,0.0,1.0,1.000000e+09,0.0,0.0,9.571069,0,0.0,1.0,./logs/fgm\Opp_bumper_v_5.csv_FFG179.5_SCD24.0...
2,185.4000,24.1000,0.8950,0.4420,0.5150,0.5850,7.2800,0.0660,1.7800,0.2050,...,0.0,1.0,1.000000e+09,0.0,0.0,9.943865,0,0.0,1.0,./logs/fgm\Opp_bumper_v_3.csv_FFG185.4_SCD24.1...
3,188.8425,25.4270,0.9078,0.3976,0.5093,0.5299,7.6821,0.0685,1.7813,0.2109,...,0.0,1.0,1.000000e+09,0.0,0.0,12.871924,0,0.0,1.0,./logs/fgm\Opp_bumper_slow_2.csv_FFG188.8425_S...
4,170.2726,26.6021,0.8619,0.4548,0.5280,0.6059,8.4557,0.0614,1.7876,0.1655,...,0.0,1.0,1.000000e+09,0.0,0.0,10.451205,0,0.0,1.0,./logs/fgm\Opp_bumper_v_4.csv_FFG170.2726_SCD2...


In [9]:
def get_first_existing(row, candidates, default=np.nan):
    for c in candidates:
        if c in row and not pd.isna(row[c]):
            return row[c]
    return default


In [10]:
def compute_safety_comfort_score(row,
                                 d_safe=0.5,
                                 d_good=0.8,
                                 track_rms_bad=1.0,
                                 w_safety=0.6,
                                 w_progress=0.3,
                                 w_comfort=0.1):
    """
    progress는 아직 모르는 상태라 safety+comfort만 보고 임시 점수 계산.
    (progress는 나중에 duration min/max로 정규화해서 반영)
    """
    # 충돌 / 비정상 종료 체크
    result = str(row.get("result", "")).upper()
    collision_flag = 0
    if "Collision" in row:
        collision_flag = int(row["Collision"])
    elif "collision_flag" in row:
        collision_flag = int(row["collision_flag"])
    
    if collision_flag == 1 or result in ("CRASH", "STUCK", "ABORT"):
        return 0.0
    
    # ----- Safety -----
    fgm_min = get_first_existing(row, ["FGM Min Dist (m)", "fgm_min_dist", "Min Dist (m)", "min_dist"], default=np.nan)
    if math.isnan(fgm_min):
        fgm_min = d_safe
    
    fgm_unsafe = get_first_existing(row, ["FGM Unsafe Ratio", "fgm_unsafe_ratio", "Unsafe Ratio", "unsafe_ratio"], default=0.0)
    if math.isnan(fgm_unsafe):
        fgm_unsafe = 0.0
    
    # 최소 거리 점수
    Sd = (fgm_min - d_safe) / (d_good - d_safe) if d_good > d_safe else 1.0
    Sd = max(0.0, min(1.0, Sd))
    
    # Unsafe Ratio 점수
    Su = 1.0 - max(0.0, min(1.0, fgm_unsafe))
    
    S_safety = 0.7 * Sd + 0.3 * Su
    
    # ----- Comfort (트랙 RMS) -----
    track_rms = get_first_existing(row, ["FGM Track RMS (m)", "fgm_track_rms"], default=0.0)
    if math.isnan(track_rms):
        track_rms = 0.0
    
    S_track = 1.0 - max(0.0, min(1.0, track_rms / track_rms_bad))
    S_comfort = S_track
    
    # progress 제외 상태에서 임시 점수 (safety+comfort 비율만 유지)
    # w_progress는 일단 무시하고, (w_safety + w_comfort)만 사용
    w_sc = w_safety + w_comfort
    Score0 = 0.0
    if w_sc > 0:
        Score0 = (w_safety * S_safety + w_comfort * S_comfort) / w_sc
    return Score0 * 100.0


In [11]:
# 1) safety/comfort 기반 임시 점수 계산
summary_df["Score_tmp"] = summary_df.apply(compute_safety_comfort_score, axis=1)

# 2) duration_raw 칼럼 준비 (Summary > 파일명 순으로)
summary_df["duration_raw"] = summary_df.apply(
    lambda r: get_first_existing(r, ["Duration (s)", "duration_from_name"], default=np.nan),
    axis=1
)

# 3) duration min/max (충돌/ABORT 제외)
valid_mask = (summary_df["Score_tmp"] > 0) & (~summary_df["duration_raw"].isna())
if valid_mask.any():
    T_min = summary_df.loc[valid_mask, "duration_raw"].min()
    T_max = summary_df.loc[valid_mask, "duration_raw"].max()
else:
    T_min, T_max = np.nan, np.nan

def compute_progress(duration, T_min, T_max):
    if math.isnan(duration) or math.isnan(T_min) or math.isnan(T_max) or T_max <= T_min:
        return 0.0
    val = (T_max - duration) / (T_max - T_min)
    return max(0.0, min(1.0, val))

summary_df["S_progress"] = summary_df["duration_raw"].apply(
    lambda t: compute_progress(t, T_min, T_max)
)

def compute_final_score(row,
                        w_safety=0.6,
                        w_progress=0.3,
                        w_comfort=0.1):
    # 충돌 / 비정상 종료면 0점
    result = str(row.get("result", "")).upper()
    collision_flag = 0
    if "Collision" in row:
        collision_flag = int(row["Collision"])
    elif "collision_flag" in row:
        collision_flag = int(row["collision_flag"])
    if collision_flag == 1 or result in ("CRASH", "STUCK", "ABORT"):
        return 0.0
    
    # safety+comfort는 Score_tmp 에 이미 들어 있음 (0~100)
    Score_tmp = row["Score_tmp"]
    # progress는 0~1
    S_progress = row["S_progress"]
    
    # 안전+컴포트 점수 비율 = (1 - w_progress)
    # 최종: Score = (1 - w_progress)*Score_tmp + w_progress*(100 * S_progress)
    Score_final = (1.0 - w_progress) * Score_tmp + w_progress * (100.0 * S_progress)
    
    # 클립
    return max(0.0, min(100.0, Score_final))

summary_df["Score"] = summary_df.apply(compute_final_score, axis=1)

# 점수 높은 순으로 보기
summary_df_sorted = summary_df.sort_values("Score", ascending=False)
summary_df_sorted.head(10)[[
    "scenario", "result",
    "FFG", "SCD", "Gap", "Bub", "Clr", "WW", "AW", "SW", "HB", "CT", "SA", "DBS", "SPD",
    "Score",
    "FGM Min Dist (m)","FGM Unsafe Ratio","FGM Track RMS (m)",
    "Duration (s)","duration_from_name"
]]


,scenario,result,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,...,CT,SA,DBS,SPD,Score,FGM Min Dist (m),FGM Unsafe Ratio,FGM Track RMS (m),Duration (s),duration_from_name
378,Opp_bumper_v_4.csv_FFG198.2499_SCD20.1949,GOAL,198.2499,20.1949,0.9002,0.4214,0.4936,0.6501,7.6198,0.0633,...,0.2156,0.4468,0.1344,4.2536,100.000000,1.000000e+09,0.0,0.0,8.90,8.90
55,Opp_bumper_v_3.csv_FFG198.2499_SCD20.1949,GOAL,198.2499,20.1949,0.9002,0.4214,0.4936,0.6501,7.6198,0.0633,...,0.2156,0.4468,0.1344,4.2536,99.795720,1.000000e+09,0.0,0.0,9.04,9.04
120,Opp_bumper_v_4.csv_FFG193.5_SCD22.6,GOAL,193.5000,22.6000,0.8500,0.4100,0.5050,0.5300,8.3000,0.0590,...,0.1780,0.4700,0.1700,4.5500,99.737354,1.000000e+09,0.0,0.0,9.08,9.08
264,Opp_bumper_v_5.csv_FFG191.0_SCD22.8,GOAL,191.0000,22.8000,0.9400,0.4500,0.5200,0.6100,8.1000,0.0580,...,0.1850,0.4800,0.1600,4.5000,99.576848,1.000000e+09,0.0,0.0,9.19,9.19
171,Opp_bumper_v_4.csv_FFG191.0_SCD22.8,GOAL,191.0000,22.8000,0.9400,0.4500,0.5200,0.6100,8.1000,0.0580,...,0.1850,0.4800,0.1600,4.5000,99.576848,1.000000e+09,0.0,0.0,9.19,9.19
294,Opp_bumper_v_2.csv_FFG198.2499_SCD20.1949,GOAL,198.2499,20.1949,0.9002,0.4214,0.4936,0.6501,7.6198,0.0633,...,0.2156,0.4468,0.1344,4.2536,99.460117,1.000000e+09,0.0,0.0,9.27,9.27
163,Opp_bumper_v_2.csv_FFG183.7312_SCD22.6077,GOAL,183.7312,22.6077,0.9218,0.4443,0.5344,0.6270,7.3401,0.0689,...,0.2150,0.4664,0.1456,4.1177,99.430934,1.000000e+09,0.0,0.0,9.29,9.29
286,Opp_bumper_v_5.csv_FFG183.7312_SCD22.6077,GOAL,183.7312,22.6077,0.9218,0.4443,0.5344,0.6270,7.3401,0.0689,...,0.2150,0.4664,0.1456,4.1177,99.430934,1.000000e+09,0.0,0.0,9.29,9.29
339,Opp_bumper_v_3.csv_FFG183.7312_SCD22.6077,GOAL,183.7312,22.6077,0.9218,0.4443,0.5344,0.6270,7.3401,0.0689,...,0.2150,0.4664,0.1456,4.1177,99.387160,1.000000e+09,0.0,0.0,9.32,9.32
325,Opp_bumper_v_3.csv_FFG166.0933_SCD23.6056,GOAL,166.0933,23.6056,0.8039,0.4086,0.5336,0.4525,7.8124,0.0652,...,0.1769,0.5431,0.1436,4.3800,99.299611,1.000000e+09,0.0,0.0,9.38,9.38


In [12]:
param_cols = [    "FFG", "SCD", "Gap", "Bub", "Clr", "WW", "AW", "SW", "HB", "CT", "SA", "DBS", "SPD"]

grouped = (
    summary_df
    .groupby(param_cols)
    .agg(
        mean_score=("Score", "mean"),
        min_score=("Score", "min"),
        max_score=("Score", "max"),
        runs=("Score", "count"),
        crashes=("Collision", "sum") if "Collision" in summary_df.columns else ("collision_flag", "sum")
    )
    .reset_index()
    .sort_values("mean_score", ascending=False)
)

grouped


,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,HB,CT,SA,DBS,SPD,mean_score,min_score,max_score,runs,crashes
15,179.5000,24.0000,0.8700,0.4200,0.5000,0.5250,7.5500,0.0640,1.7500,0.2000,0.5400,0.1500,4.3000,96.996248,93.915370,99.022374,7,0
22,183.4000,25.3000,0.8900,0.4550,0.5350,0.5900,7.6000,0.0610,1.7000,0.1980,0.5050,0.1520,4.4000,96.825320,94.863813,98.292802,7,0
32,186.8000,24.4000,0.8900,0.4450,0.5150,0.5850,7.6500,0.0620,1.7800,0.2060,0.5050,0.1580,4.4500,96.816982,94.353113,98.570039,7,0
35,187.3000,23.1000,0.9050,0.4650,0.5150,0.5800,7.2500,0.0670,1.8200,0.2150,0.4950,0.1420,4.1500,96.806907,87.538911,99.168288,6,0
21,182.8019,23.9747,0.9218,0.4384,0.4825,0.5297,7.7049,0.0687,1.7218,0.1961,0.5165,0.1679,4.4772,96.723180,92.806420,98.818093,7,0
23,183.7312,22.6077,0.9218,0.4443,0.5344,0.6270,7.3401,0.0689,1.8277,0.2150,0.4664,0.1456,4.1177,96.527237,87.451362,99.430934,8,0
44,191.0000,22.8000,0.9400,0.4500,0.5200,0.6100,8.1000,0.0580,1.6000,0.1850,0.4800,0.1600,4.5000,96.315661,86.415370,99.576848,8,0
8,176.9000,26.0000,0.9150,0.4700,0.5450,0.6000,6.9500,0.0700,1.9800,0.2100,0.5150,0.1400,4.1000,96.104086,92.893969,98.000973,7,0
4,170.5157,24.6237,0.9552,0.4067,0.5363,0.5671,7.4008,0.0659,1.7732,0.2192,0.4988,0.1435,4.3212,95.950875,86.181907,98.788911,8,0
16,179.8138,27.0754,0.8533,0.3856,0.5072,0.5342,8.3709,0.0683,1.8000,0.2234,0.4886,0.1423,3.9813,95.933852,93.900778,97.402724,6,0


In [13]:
import numpy as np
import pandas as pd

# FFG = fgm_fov_angle
# SCD = fgm_speed_check_fov_deg
param_cols = [
    "FFG", "SCD",   # 새로 추가된 두 파라미터
    "Gap", "Bub", "Clr", "WW", "AW", "SW", "HB", "CT", "SA", "DBS", "SPD"
]

# 각 파라미터 튜닝 범위 (대략 이전에 쓰던 값 기준으로 적당히 설정)
param_bounds = {
    "FFG": (140.0, 210.0),   # fov angle [deg]
    "SCD": (10.0,  45.0),    # speed check fov [deg]

    "Gap": (0.8,  2.0),
    "Bub": (0.3,  0.8),
    "Clr": (0.4,  0.8),
    "WW":  (0.2,  1.0),
    "AW":  (2.0, 10.0),
    "SW":  (0.02, 0.2),
    "HB":  (0.0,  4.0),
    "CT":  (0.15, 0.5),
    "SA":  (0.2,  0.8),
    "DBS": (0.05, 0.3),
    "SPD": (2.5,  6.0),
}


In [14]:
rng = np.random.default_rng(42)  # 필요하면 42 말고 다른 숫자로 바꿔도 됨

def propose_new_params_simple(
    grouped: pd.DataFrame,
    param_cols,
    param_bounds,
    n_seeds=3,       # mean_score 상위 몇 세트를 seed로 쓸지
    per_seed=10,     # seed 하나당 몇 개 후보를 생성할지
    noise_scale=0.08 # ±8% 정도 multiplicative noise
):
    """
    grouped: FFG,SCD,Gap~SPD, mean_score 등이 들어있는 DataFrame
    param_cols: 튜닝할 파라미터 컬럼 이름 리스트
    param_bounds: 각 파라미터 별 (min,max) 딕셔너리
    """
    # 1) mean_score 높은 순서대로 상위 n_seeds 뽑기
    top = grouped.sort_values("mean_score", ascending=False).head(n_seeds)
    
    new_rows = []
    for _, seed in top.iterrows():
        base = seed[param_cols].astype(float).values  # seed 파라미터 벡터
        
        for _ in range(per_seed):
            # multiplicative gaussian noise: θ' = θ * (1 + N(0, noise_scale))
            noise = rng.normal(loc=0.0, scale=noise_scale, size=len(param_cols))
            cand = base * (1.0 + noise)
            
            # 각 파라미터 별 bounds 안으로 자르기
            cand_clipped = []
            for val, name in zip(cand, param_cols):
                lo, hi = param_bounds[name]
                cand_clipped.append(float(np.clip(val, lo, hi)))
            
            new_rows.append(dict(zip(param_cols, cand_clipped)))
    
    new_df = pd.DataFrame(new_rows)
    
    # 2) 이미 실험한 파라미터 세트와 완전히 같은 조합은 제거 (중복 방지)
    merged = new_df.merge(grouped[param_cols], on=param_cols, how="left", indicator=True)
    new_unique = merged[merged["_merge"] == "left_only"][param_cols].reset_index(drop=True)
    
    return new_unique


In [15]:
new_candidates = propose_new_params_simple(
    grouped,
    param_cols=param_cols,
    param_bounds=param_bounds,
    n_seeds=3,      # mean_score 상위 3개를 seed로
    per_seed=10,    # seed 하나당 10개씩 → 총 30개 후보
    noise_scale=0.08
)

print("새로 제안된 파라미터 개수:", len(new_candidates))
new_candidates.head()


새로 제안된 파라미터 개수: 30


,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,HB,CT,SA,DBS,SPD
0,183.875737,22.003231,0.922231,0.451603,0.421959,0.470308,7.627216,0.062381,1.747648,0.186351,0.577990,0.159334,4.322715
1,195.687184,24.897618,0.810193,0.432390,0.461645,0.561895,7.519845,0.063054,1.654670,0.219561,0.533324,0.144860,4.178866
2,187.143960,24.701653,0.898726,0.434476,0.585666,0.507931,7.240605,0.059833,1.836237,0.218064,0.535077,0.139918,4.016378
3,188.842512,25.427048,0.907804,0.397639,0.509286,0.529901,7.682088,0.068462,1.781303,0.210863,0.542919,0.153469,4.517163
4,158.575242,23.386231,0.837262,0.398534,0.488994,0.587788,7.027038,0.068958,1.514398,0.194642,0.547031,0.157035,4.544662


In [16]:
def df_to_params_list_strings(df):
    """
    new_candidates DataFrame(FFG,SCD,Gap~SPD 13개) → bash params_list용 문자열 리스트
    """
    lines = []
    for _, row in df.iterrows():
        vals = [
            row["Gap"],
            row["Bub"],
            row["FFG"],
            row["SCD"],
            row["Clr"],
            row["WW"],
            row["AW"],
            row["SW"],
            row["HB"],
            row["CT"],
            row["SA"],
            row["DBS"],
            row["SPD"],
            
        ]
        s = " ".join(f"{v:.4f}" for v in vals)
        lines.append(f"\"{s}\"")
    return lines

param_strings = df_to_params_list_strings(new_candidates)

print("params_list=(")
for s in param_strings:
    print("    " + s)
print(")")


params_list=(
    "0.9222 0.4516 183.8757 22.0032 0.4220 0.4703 7.6272 0.0624 1.7476 0.1864 0.5780 0.1593 4.3227"
    "0.8102 0.4324 195.6872 24.8976 0.4616 0.5619 7.5198 0.0631 1.6547 0.2196 0.5333 0.1449 4.1789"
    "0.8987 0.4345 187.1440 24.7017 0.5857 0.5079 7.2406 0.0598 1.8362 0.2181 0.5351 0.1399 4.0164"
    "0.9078 0.3976 188.8425 25.4270 0.5093 0.5299 7.6821 0.0685 1.7813 0.2109 0.5429 0.1535 4.5172"
    "0.8373 0.3985 158.5752 23.3862 0.4890 0.5878 7.0270 0.0690 1.5144 0.1946 0.5470 0.1570 4.5447"
    "0.8378 0.4488 190.8925 23.3304 0.4923 0.4714 6.8655 0.0593 1.8196 0.2023 0.5698 0.1449 4.3545"
    "0.9018 0.3978 188.4835 23.4061 0.4855 0.5090 6.8277 0.0665 1.6843 0.2002 0.5608 0.1554 4.5289"
    "0.8645 0.3633 178.0857 23.1873 0.4421 0.4694 6.9477 0.0660 1.6232 0.1939 0.5961 0.1457 4.5537"
    "0.8039 0.4086 166.0933 23.6056 0.5336 0.4525 7.8124 0.0652 1.6668 0.1769 0.5431 0.1436 4.3800"
    "0.8533 0.3856 179.8138 27.0754 0.5072 0.5342 8.3709 0.0683 1.8000 0.2234 0.4886 0